In [1]:
import cv2
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization
import torch
import numpy as np
import os

# Ví dụ feature extraction

## HOG + Openface

In [2]:
sample_path = "/Users/trananhvu/Documents/CV/CV_internship/face_recognition/step_by_step_face_recognition/sample/preprocess"

In [3]:
image = cv2.imread(os.path.join(sample_path, "preprocess_img_hog.jpg"))

In [4]:
# Tham khảo trên: https://stackoverflow.com/questions/70492290/how-to-make-a-face-recognitionidentification-and-verification-program
openface = cv2.dnn.readNetFromTorch("/Users/trananhvu/Documents/CV/CV_internship/model/nn4.small2.v1.t7")

In [5]:
blob = cv2.dnn.blobFromImage(image, 1./255, (96,96), (0,0,0), True, False)
openface.setInput(blob)
feature = openface.forward()
feature.shape

(1, 128)

## MTCNN + Facenet

In [6]:
image = cv2.imread(os.path.join(sample_path, "preprocess_img_mtcnn.jpg"))
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [7]:
# def fixed_image_standardization(image_tensor):
#     processed_tensor = (image_tensor - 127.5) / 128.0
#     return processed_tensor
# def trans(img):
#     transform = transforms.Compose([
#             transforms.ToTensor(),
#             fixed_image_standardization,
#         ])
#     return transform(img)

In [8]:
facenet = InceptionResnetV1(pretrained='vggface2', dropout_prob=0.5, num_classes=6, classify=False).eval()

In [9]:
# ta normalize ảnh từ [-1, 1] để phù hợp với input của facenet
blob = cv2.dnn.blobFromImage(image, 1./128, (160,160), (127.5, 127.5, 127.5), False, False)
feature = facenet(torch.tensor(blob))
# feature = facenet(torch.tensor(image).reshape(image.shape[2], image.shape[0], image.shape[1]).float().unsqueeze(0))
# feature = facenet(trans(image).reshape(image.shape[2], image.shape[0], image.shape[1]).unsqueeze(0))
feature.shape

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


torch.Size([1, 512])

# Feature Extraction các ảnh mẫu

In [10]:
# Path setting
preprocess_path = "/Users/trananhvu/Documents/CV/data/preprocess"
feature_path = "/Users/trananhvu/Documents/CV/data/feature"
source_hog_path = os.path.join(preprocess_path, "hog")
save_parent_hog_openface_path = os.path.join(feature_path, "hog_openface")
source_mtcnn_path = os.path.join(preprocess_path, "mtcnn")
save_parent_mtcnn_facenet_path = os.path.join(feature_path, "mtcnn_facenet")

## HOG + Openface

In [11]:
# Ta sẽ lấy vector đặc trưng sử dụng facenet
def get_openface_feature(image_path, save_path):
    image = cv2.imread(image_path)
    blob = cv2.dnn.blobFromImage(image, 1./255, (96,96), (0,0,0), True, False)
    openface.setInput(blob)
    feature = openface.forward()
    np.save(os.path.join(save_path, os.path.basename(image_path)[:-4]+".npy"), feature)

In [12]:
for i in os.listdir(source_hog_path):
    if i==".DS_Store":
        continue
    print(i)
    image_source_path = os.path.join(source_hog_path, i)
    for j in os.listdir(image_source_path):
        if j==".DS_Store":
            continue
        print("Image: "+j)
        if not os.path.exists(os.path.join(save_parent_hog_openface_path, i)):
            os.mkdir(os.path.join(save_parent_hog_openface_path, i))
        get_openface_feature(os.path.join(image_source_path, j), os.path.join(save_parent_hog_openface_path, i))

Robert Downey Jr
Image: Robert Downey Jr15_4151.jpg
Image: Robert Downey Jr51_4282.jpg
Image: Robert Downey Jr243_4256.jpg
Image: Robert Downey Jr122_4100.jpg
Image: Robert Downey Jr50_4281.jpg
Image: Robert Downey Jr105_4087.jpg
Image: Robert Downey Jr242_4255.jpg
Image: Robert Downey Jr118_4096.jpg
Image: Robert Downey Jr61_4288.jpg
Image: Robert Downey Jr119_4097.jpg
Image: Robert Downey Jr50_4280.jpg
Image: Robert Downey Jr42_4273.jpg
Image: Robert Downey Jr20_4192.jpg
Image: Robert Downey Jr85_4306.jpg
Image: Robert Downey Jr215_4206.jpg
Image: Robert Downey Jr221_4210.jpg
Image: Robert Downey Jr170_4167.jpg
Image: Robert Downey Jr234_4245.jpg
Image: Robert Downey Jr65_4292.jpg
Image: Robert Downey Jr167_4161.jpg
Image: Robert Downey Jr92_4314.jpg
Image: Robert Downey Jr93_4315.jpg
Image: Robert Downey Jr166_4160.jpg
Image: Robert Downey Jr221_4211.jpg
Image: Robert Downey Jr128_4105.jpg
Image: Robert Downey Jr99_4316.jpg
Image: Robert Downey Jr112_4091.jpg
Image: Robert Downey Jr

## MTCNN + Facenet

In [13]:
# Ta sẽ lấy vector đặc trưng sử dụng facenet
def get_facenet_feature(image_path, save_path):
    image = cv2.imread(image_path)
    blob = cv2.dnn.blobFromImage(image, 1./128, (160,160), (127.5, 127.5, 127.5), False, False)
    with torch.no_grad():
        feature = facenet(torch.tensor(blob))
        # feature = facenet(torch.tensor(image).reshape(image.shape[2], image.shape[0], image.shape[1]).float().unsqueeze(0))
        # feature = facenet(trans(image).reshape(image.shape[2], image.shape[0], image.shape[1]).float().unsqueeze(0))
        feature = feature.detach().numpy()
        np.save(os.path.join(save_path, os.path.basename(image_path)[:-4]+".npy"), feature)

In [14]:
for i in os.listdir(source_mtcnn_path):
    if i==".DS_Store":
        continue
    print(i)
    image_source_path = os.path.join(source_mtcnn_path, i)
    for j in os.listdir(image_source_path):
        if j==".DS_Store":
            continue
        print("Image: "+j)
        if not os.path.exists(os.path.join(save_parent_mtcnn_facenet_path, i)):
            os.mkdir(os.path.join(save_parent_mtcnn_facenet_path, i))
        get_facenet_feature(os.path.join(image_source_path, j), os.path.join(save_parent_mtcnn_facenet_path, i))

Robert Downey Jr
Image: Robert Downey Jr15_4151.jpg
Image: Robert Downey Jr51_4282.jpg
Image: Robert Downey Jr243_4256.jpg
Image: Robert Downey Jr50_4281.jpg
Image: Robert Downey Jr105_4087.jpg
Image: Robert Downey Jr242_4255.jpg
Image: Robert Downey Jr118_4096.jpg
Image: Robert Downey Jr61_4288.jpg
Image: Robert Downey Jr119_4097.jpg
Image: Robert Downey Jr50_4280.jpg
Image: Robert Downey Jr42_4273.jpg
Image: Robert Downey Jr20_4192.jpg
Image: Robert Downey Jr85_4306.jpg
Image: Robert Downey Jr215_4206.jpg
Image: Robert Downey Jr221_4210.jpg
Image: Robert Downey Jr170_4167.jpg
Image: Robert Downey Jr234_4245.jpg
Image: Robert Downey Jr65_4292.jpg
Image: Robert Downey Jr167_4161.jpg
Image: Robert Downey Jr92_4314.jpg
Image: Robert Downey Jr93_4315.jpg
Image: Robert Downey Jr166_4160.jpg
Image: Robert Downey Jr221_4211.jpg
Image: Robert Downey Jr99_4316.jpg
Image: Robert Downey Jr112_4091.jpg
Image: Robert Downey Jr131_4108.jpg
Image: Robert Downey Jr128_4107.jpg
Image: Robert Downey Jr